In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from loguru import logger
from dotenv import load_dotenv
from pydantic import BaseModel
from tqdm.notebook import tqdm

import numpy as np # required for the scikit-learn pipeline to work
import pandas as pd
import plotly.express as px
import mlflow

load_dotenv()

sys.path.insert(0, '..')

from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = '020-baseline-popular-books'
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = 'user_id'
    item_col: str = 'parent_asin'
    rating_col: str = 'rating'
    
    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        
        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            import mlflow

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self
    
args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-18 22:25:18.137 | INFO     | __main__:init:28 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 020-baseline-popular-books...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "020-baseline-popular-books",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/020-baseline-popular-books",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Prep data

In [4]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [5]:
user_col = args.user_col
item_col = args.item_col
rating_col = args.rating_col

# Implement

In [6]:
popular_items_df = (
    train_df
    .groupby(item_col, as_index=False)
    .size()
    .assign(
        rec_ranking=lambda df: df['size'].rank(method='first', ascending=False).astype(int)
    )
    .sort_values(['rec_ranking'], ascending=[True])
    .head(args.top_K)
)

popular_items_df

,parent_asin,size,rec_ranking
19921,B00L9B7IKE,35,1
18720,B006LSZECO,27,2
19433,B00DPM7TIG,23,3
20417,B00YTXTIDO,20,4
21135,B01KXQ8SS6,20,5
...,...,...,...
20170,B00R04GCMY,7,96
20210,B00SEFAIRI,7,97
20221,B00SLWQGUM,7,98
20279,B00UCLPCE6,7,99


In [7]:
recommendations_df = (
    val_df[[user_col]].drop_duplicates().assign(tmp=1)
    .pipe(lambda df: pd.merge(
        df,
        popular_items_df.assign(tmp=1),
        on='tmp',
        how='left'
    ))
    .rename(columns={'size': 'score'})
    [[user_col, item_col, 'score', 'rec_ranking']]
)
    
recommendations_df

,user_id,parent_asin,score,rec_ranking
0,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,B00L9B7IKE,35,1
1,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,B006LSZECO,27,2
2,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,B00DPM7TIG,23,3
3,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,B00YTXTIDO,20,4
4,AHXBL3QDWZGJYH7A5CMPFNUPMF7Q,B01KXQ8SS6,20,5
...,...,...,...,...
59295,AFM4K7CAFB2KE6BHWQSS7KEHTWLA,B00R04GCMY,7,96
59296,AFM4K7CAFB2KE6BHWQSS7KEHTWLA,B00SEFAIRI,7,97
59297,AFM4K7CAFB2KE6BHWQSS7KEHTWLA,B00SLWQGUM,7,98
59298,AFM4K7CAFB2KE6BHWQSS7KEHTWLA,B00UCLPCE6,7,99


# Evaluate

## Ranking metrics

In [8]:
from src.eval import create_label_df, merge_recs_with_target

In [9]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
294,AFG6YQ3GOY7TVFKQ3SKDVS6Q6RDQ,B07R3QYGHY,4.0,1.0
123756,AEAH73R5WSYAYJF36EFVA6KO5YEA,B089Q37QPL,5.0,1.0
137190,AGFYZJ7SQR6GBVKHAA7AZWDHE3BA,B00M70YWKK,2.0,1.0
251890,AEIJXPELONBMK2ETYVX7OWYLSCYQ,B0036S4CIO,4.0,1.0
32134,AEKIZAGIGDREPS2C37A5OENBYIFA,1585365076,5.0,1.0
...,...,...,...,...
223203,AH2AFLWCKAMIQUMEXQU4BDY3A5XQ,B06W56QBHM,2.0,10.0
261755,AGGEA3XF6BPM4FTQ4NIU7AHQ7LJA,B000FC29HK,2.0,11.0
97266,AHKJ3ITSMUU4DAMJHDLPXBH4QJRA,B07QS6KZFN,5.0,11.0
97265,AHKJ3ITSMUU4DAMJHDLPXBH4QJRA,B086DQQNSD,5.0,12.0


In [10]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_id,parent_asin,score,rec_ranking,rating,rating_rank
31,AE25MJXNW6C5SBWOTFT3SMXIIZMQ,B00L9B7IKE,35.0,1,0,NaN
16,AE25MJXNW6C5SBWOTFT3SMXIIZMQ,B006LSZECO,27.0,2,0,NaN
22,AE25MJXNW6C5SBWOTFT3SMXIIZMQ,B00DPM7TIG,23.0,3,0,NaN
44,AE25MJXNW6C5SBWOTFT3SMXIIZMQ,B00YTXTIDO,20.0,4,0,NaN
65,AE25MJXNW6C5SBWOTFT3SMXIIZMQ,B01KXQ8SS6,20.0,5,0,NaN
...,...,...,...,...,...,...
60127,AHZSREYRPFC3YQ5XHVVNC5LXIMBA,B00SLWQGUM,7.0,98,0,NaN
60129,AHZSREYRPFC3YQ5XHVVNC5LXIMBA,B00UCLPCE6,7.0,99,0,NaN
60151,AHZSREYRPFC3YQ5XHVVNC5LXIMBA,B01FQLZSPY,7.0,100,0,NaN
60188,AHZSREYRPFC3YQ5XHVVNC5LXIMBA,B084MHZRGG,NaN,101,5,2.0


### Visualize

In [11]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metrics import PrecisionTopKMetric
from evidently.metrics import RecallTopKMetric
from evidently.metrics import FBetaTopKMetric
from evidently.metrics import NDCGKMetric
from evidently.metrics import PersonalizationMetric
import warnings

warnings.filterwarnings(
    action='ignore',
    category=FutureWarning,
    module=r'evidently.metrics.recsys.precision_recall_k'
)

from src.viz import color_scheme

In [12]:
column_mapping = ColumnMapping(
    recommendations_type='rank',
    target=rating_col,
    prediction='rec_ranking',
    item_id=item_col,
    user_id=user_col
)

report = Report(metrics=[
    NDCGKMetric(k=args.top_k),
    RecallTopKMetric(k=args.top_K),
    PrecisionTopKMetric(k=args.top_k),
    FBetaTopKMetric(k=args.top_k),
    PersonalizationMetric(k=args.top_k),
], options=[color_scheme])

report.run(
    reference_data=None,
    current_data=eval_df,
    column_mapping=column_mapping
)

evidently_report_fp = f"{args.notebook_persist_dp}/evidently_report.html"
os.makedirs(args.notebook_persist_dp, exist_ok=True)
report.save_html(evidently_report_fp)

if args.log_to_mlflow:
    mlflow.log_artifact(evidently_report_fp)
    for metric_result in report.as_dict()['metrics']:
        metric = metric_result['metric']
        if metric == 'PersonalizationMetric':
            metric_value = float(metric_result['result']['current_value'])
            mlflow.log_metric(f"val_{metric}", metric_value)
            continue
        result = metric_result['result']['current'].to_dict()
        for kth, metric_value in result.items():
            mlflow.log_metric(f"val_{metric}_at_k_as_step", metric_value, step=kth)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


# Clean up

In [13]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/18 22:25:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run 020-baseline-popular-books at: http://localhost:5003/#/experiments/1/runs/c415c61519724e7a9f15049bd543be7d.
2024/09/18 22:25:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.
